# EDA

- Goal: Design a model to predict each matchup outcome for MLB games
- Data Source: https://baseballsavant.mlb.com/statcast_search
- Things to look into
    - What predicts matchout outcome before the initial pitch?
        - pitcher, batter (career batting average, batting average of batters X facing pitcher Y), 
        - left-handed vs right-handed?
        - batters' speed and position (probably correlated with probabilities at the plate)
        - weather (temperature, humidity, wind direction)
        - Ballpark characteristics (away vs home, not all stadiums are alike)
        - Specific game situation (players on-base, bottom 9th and 2 outs so walk is unlikely), 
        - modern statistics (batted-ball exit velocity?
        - Starting with the predictors here: https://www.baseballprospectus.com/news/article/59993/singlearity-using-a-neural-network-to-predict-the-outcome-of-plate-appearances/#_ftn12
    - Predictors: events
        - 7 events?
        - out, single, double, triple, homerun, walk, hit-by-pitch
    - Data validation (downloaded data vs mlb.com)
    - understands each field/column
    - eda
        - at bat vs plate appearance
        - power hitter vs contact hitter
        - 
    - preprocessing pipeline
    - simple model
    - metrics
    - fine-tune: feature selection / model selection / HP search
    - Application
        - Situational analysis: what to do for a certain situation for the coaches
        - Daily fantasy: Simulate games for all team matchup, then aggregate the average stats for each player
    
    
    
    

In [1]:
%load_ext autoreload
%autoreload 2

In [270]:
import sys
from datetime import date, timedelta

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.append('/Users/allenchen/projects/baseball-analytics/src')
from modeling.pipeline import data_pipeline

pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 400)

In [159]:
# config

# gives game state data
#ANGELS_PITCHER_DATA_PATH = "../data/20230418_angels_pitcher_stats.csv"
ANGELS_BATTER_DATA_PATH = "../data/20230418_angels_batter_stats.csv"
ANGELS_BATTER_2022_PATH = "../data/angels_batter_2022_season.csv"
start_of_season = datetime(2023, 3, 30)

In [160]:
#angels_pitcher_data = pd.read_csv(ANGELS_PITCHER_DATA_PATH)
angels_batter_data = pd.read_csv(ANGELS_BATTER_DATA_PATH)
angels_batter_2022_data = pd.read_csv(ANGELS_BATTER_2022_PATH)

In [161]:
angels_batter_data_2022_2023 = pd.concat([angels_batter_data, angels_batter_2022_data])

In [162]:
#print(angels_pitcher_data.shape)
print(angels_batter_data_2022_2023.shape)

(25610, 92)


In [163]:
angels_batter_data_2022_2023.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
0,SL,2023-04-17,83.6,-2.11,5.48,"Neto, Zach",687263,678394,NaN,foul,NaN,NaN,NaN,NaN,9,Zach Neto flies out to right fielder Raimel Tapia.,R,R,R,BOS,LAA,S,NaN,NaN,2,1,2023,0.25,0.13,0.42,2.02,NaN,571875.0,592273.0,2,3,Top,NaN,NaN,NaN,NaN,624512,NaN,NaN,5.359175,-121.637612,-1.805560,1.425656,23.284948,-30.747685,3.37,1.53,2.0,65.4,-46.0,83.7,2487.0,6.3,718540,678394,624512,671213,624414,646240,571771,608701,680776,606132,54.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,4,Slider,1,5,5,1,5,1,5,1,Strategic,Standard,56.0,0.000,-0.147
1,FF,2023-04-17,95.0,-1.14,5.96,"Ward, Taylor",621493,676710,strikeout,swinging_strike,NaN,NaN,NaN,NaN,11,Taylor Ward strikes out swinging.,R,R,R,BOS,LAA,S,2.0,NaN,0,2,2023,-0.90,1.48,-0.45,4.04,NaN,NaN,NaN,2,9,Top,NaN,NaN,NaN,NaN,657136,NaN,NaN,3.801963,-138.289353,-2.679609,-12.345344,31.494520,-12.839856,3.50,1.74,NaN,NaN,NaN,94.9,2557.0,6.3,718540,676710,657136,671213,624414,646240,571771,608701,680776,606132,54.24,NaN,NaN,0.0,1.0,0.0,0.0,NaN,73,3,4-Seam Fastball,3,5,5,3,5,3,5,3,Standard,Standard,204.0,0.003,-0.055
2,FC,2023-04-17,88.5,-1.28,5.94,"Ward, Taylor",621493,676710,NaN,foul,NaN,NaN,NaN,NaN,5,Taylor Ward strikes out swinging.,R,R,R,BOS,LAA,S,NaN,NaN,0,1,2023,0.12,0.49,0.19,2.74,NaN,NaN,NaN,2,9,Top,NaN,NaN,NaN,NaN,657136,NaN,NaN,3.341144,-128.939199,-2.808038,0.675659,25.944013,-26.359210,3.50,1.74,200.0,71.4,49.0,88.8,2310.0,6.4,718540,676710,657136,671213,624414,646240,571771,608701,680776,606132,54.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73,2,Cutter,3,5,5,3,5,3,5,3,Standard,Standard,158.0,0.000,-0.023
3,SL,2023-04-17,83.9,-1.99,5.47,"Neto, Zach",687263,678394,NaN,ball,NaN,NaN,NaN,NaN,14,Zach Neto flies out to right fielder Raimel Tapia.,R,R,R,BOS,LAA,B,NaN,NaN,1,1,2023,0.23,-0.05,1.50,1.15,NaN,571875.0,592273.0,2,3,Top,NaN,NaN,NaN,NaN,624512,NaN,NaN,7.596845,-121.921568,-3.459481,0.792663,24.037741,-32.155512,3.67,1.63,NaN,NaN,NaN,83.8,2414.0,6.3,718540,678394,624512,671213,624414,646240,571771,608701,680776,606132,54.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,3,Slider,1,5,5,1,5,1,5,1,Strategic,Standard,50.0,0.000,0.056
4,SL,2023-04-17,84.5,-1.84,5.56,"Neto, Zach",687263,678394,NaN,ball,NaN,NaN,NaN,NaN,14,Zach Neto flies out to right fielder Raimel Tapia.,R,R,R,BOS,LAA,B,NaN,NaN,0,1,2023,0.49,0.07,1.12,2.32,NaN,571875.0,592273.0,2,3,Top,NaN,NaN,NaN,NaN,624512,NaN,NaN,5.890274,-122.986290,-1.299135,3.734734,25.583836,-31.392505,3.70,1.66,NaN,NaN,NaN,84.3,2484.0,6.2,718540,678394,624512,671213,624414,646240,571771,608701,680776,606132,54.26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,2,Slider,1,5,5,1,5,1,5,1,Strategic,Standard,50.0,0.000,0.030


## Planned Features

### Game State
- Outs, [done]
- inning, [done]
- net score, [done]
- 1B occupied, [done]
- 2B occupied, [done]
- 3B occupied, [done]
- pitcher pitch number, [not available]
- top/bot of inning, [done]
- days since start of season [done]
- temperature at game start time [need weather data]

### Batter 365 day moving average
- Rates per PA for: 1B, 2B, 3B, HR, BB, IBB, HBP, GDP, SO. SF, SH
- Avg WOBA
- Max exit velocity, avg exit velocity
- # of plate appearances
- Platoon wOBA, # of platoon plate appearances
- Relative park factor hits



In [391]:
# all given columns
# data dictionary: https://baseballsavant.mlb.com/csv-docs
game_state_cols_names = [
    #'pitch_type', 
    'game_date', 
    #'release_speed',
    #'release_pos_x',
    #'release_pos_z', 
    #'player_name', # field tied to the search query
    'batter',
    'pitcher',
    'events',
    'description', 
    #'spin_dir', 
    #'spin_rate_deprecated',
    #'break_angle_deprecated', 
    #'break_length_deprecated', 
    #'zone', 
    #'des',
    'game_type', # ensure regular season only
    'stand', # side of the plate batter is standing
    'p_throws', # Hand pitcher throws with
    #'home_team', 
    #'away_team', 
    #'type', # short hand of pitch result, B=ball, S=strike, X=in play
    #'hit_location', # position of first fielder to touch the ball
    #'bb_type', # Batted ball type, ground_ball, line_drive, fly_ball, popup.
    #'balls', # pre-pitch number of balls in count
    #'strikes', # pre-pitch number of strikes in count
    #'game_year',
    #'pfx_x', 
    #'pfx_z', 
    #'plate_x', 
    #'plate_z', 
    'on_3b', # Pre-pitch MLB Player Id of Runner on 3B.
    'on_2b', # Pre-pitch MLB Player Id of Runner on 2B.
    'on_1b', # Pre-pitch MLB Player Id of Runner on 1B.
    'outs_when_up', # Pre-pitch number of outs.
    'inning', # Pre-pitch inning number.
    'inning_topbot', # Pre-pitch top or bottom of inning.
    #'hc_x', 
    #'hc_y', 
    #'tfs_deprecated', 
    #'tfs_zulu_deprecated', 
    #'fielder_2', 
    #'umpire', 
    #'sv_id',
    #'vx0', # velocity
    #'vy0', 
    #'vz0', 
    #'ax', # acceleration
    #'ay', 
    #'az',
    #'sz_top', 
    #'sz_bot',
    #'hit_distance_sc', # Projected hit distance of the batted ball.
    #'launch_speed', 
    #'launch_angle', 
    #'effective_speed', # Derived speed based on the the extension of the pitcher's release.
    #'release_spin_rate', # Spin rate of pitch tracked by Statcast.
    #'release_extension', # Release extension of pitch in feet as tracked by Statcast.
    'game_pk', # unique id for the game
    #'pitcher.1',
    #'fielder_2.1', # Player Id for catcher
    #'fielder_3', # Player id for 1B
    #'fielder_4', # 2B
    #'fielder_5', # 3B
    #'fielder_6', # SS
    #'fielder_7', # LF
    #'fielder_8', # CF 
    #'fielder_9', # RF
    #'release_pos_y', 
    #'estimated_ba_using_speedangle', 
    #'estimated_woba_using_speedangle',
    #'woba_value', # weighted on base average wOBA value based on result of play.
    #'woba_denom', 
    #'babip_value', # Batting average on Balls in Play value based on result of play. BABIP=(H - HR)/(AB - K - HR + SF)
    #'iso_value', # ISO value based on result of play. ISO (isolated power) = (1x2B + 2x3B + 3xHR) / At-bats OR Slugging percentage - Batting average
    #'launch_speed_angle', # Launch speed/angle zone based on launch angle and exit velocity.
    #'at_bat_number', # Plate appearance number of the game.
    #'pitch_number', # Total pitch number of the plate appearance.
    #'pitch_name', 
    #'home_score', 
    #'away_score', 
    'bat_score', 
    'fld_score', 
    #'post_away_score',
    #'post_home_score', 
    #'post_bat_score', 
    #'post_fld_score',
    #'if_fielding_alignment', # Infield fielding alignment at the time of the pitch.
    #'of_fielding_alignment', # Outfield fielding alignment at the time of the pitch.
    #'spin_axis',
    #'delta_home_win_exp', # The change in Win Expectancy before the Plate Appearance and after the Plate Appearance
    #'delta_run_exp' # The change in Run Expectancy before the Pitch and after the Pitch
]

In [392]:
# ETL (Transform)
# get necesssary data
data = angels_batter_data_2022_2023[game_state_cols_names].copy()
data = data[data["events"].notnull()]
data.reset_index(drop=True, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6575 entries, 0 to 6574
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   game_date      6575 non-null   object 
 1   batter         6575 non-null   int64  
 2   pitcher        6575 non-null   int64  
 3   events         6575 non-null   object 
 4   description    6575 non-null   object 
 5   game_type      6575 non-null   object 
 6   stand          6575 non-null   object 
 7   p_throws       6575 non-null   object 
 8   on_3b          595 non-null    float64
 9   on_2b          1139 non-null   float64
 10  on_1b          1972 non-null   float64
 11  outs_when_up   6575 non-null   int64  
 12  inning         6575 non-null   int64  
 13  inning_topbot  6575 non-null   object 
 14  game_pk        6575 non-null   int64  
 15  bat_score      6575 non-null   int64  
 16  fld_score      6575 non-null   int64  
dtypes: float64(3), int64(7), object(7)
memory usage: 873

In [393]:
data.shape

(6575, 17)

In [394]:
data.columns

Index(['game_date', 'batter', 'pitcher', 'events', 'description', 'game_type',
       'stand', 'p_throws', 'on_3b', 'on_2b', 'on_1b', 'outs_when_up',
       'inning', 'inning_topbot', 'game_pk', 'bat_score', 'fld_score'],
      dtype='object')

In [395]:
# clean target column

from collections import defaultdict
events_mapping_dict = {
    "field_out": "fo", 
    "strikeout": "so", 
    "single": "s", 
    "double": "d", 
    "triple": "t", 
    "walk": "w", 
    "home_run": "hr", 
    "force_out": "fo", 
    "grounded_into_double_play": "dp", 
    "double_play": "dp", 
    "hit_by_pitch": "e", 
    "field_error": "e", 
    "sac_fly": "sf", 
    "sac_bunt": "sh", 
    "fielders_choice": "fc", 
    "caught_stealing_2b": "delete", 
    "fielders_choice_out": "fo", 
    "strikeout_double_play": "so", 
    "catcher_interf": "e", 
    "triple_play": "tp", 
    "pickoff_1b": "delete", 
}
events_mapping = defaultdict(lambda: "delete", events_mapping_dict)
data["events"] = data["events"].map(events_mapping)
data.drop(data[data["events"] == "delete"].index, inplace=True)

In [396]:
data.events.value_counts()

events
fo    2704
so    1679
s      915
w      486
d      239
hr     209
dp     127
e      102
t       31
sf      30
sh      25
fc      16
tp       1
Name: count, dtype: int64

### Prepare Moving average

In [274]:
player_subset = data.loc[(data["batter"] == 660271), ["game_date", "events"]]
player_subset.head()

,game_date,events
1,2023-04-17,fo
16,2023-04-17,s
20,2023-04-17,e
34,2023-04-17,s
37,2023-04-17,fo


In [389]:
datetime(2022, 4, 7) + timedelta(365)

datetime.datetime(2023, 4, 7, 0, 0)

In [390]:

# start is the first day of the training dataset, end is the last day of the training dataset
t_index = pd.DatetimeIndex(pd.date_range(start=datetime(2022, 4, 7), end=datetime(2023, 4, 17), freq="1d"))

player_subset["game_date"] = pd.to_datetime(player_subset["game_date"])
player_subset["at_bat_count"] = 1.0
at_bat_ts = (
    player_subset[["game_date", "at_bat_count"]]
    .groupby(["game_date"])
    .sum()
    .sort_index(ascending=True)
    .resample("1D")
    .mean()
    .reindex(t_index) # most players dont have stats all the way back to the start date of the training dataset
    .fillna(0)
)
display(at_bat_ts.tail())

player_event_subset = player_subset[player_subset["events"] == 's']
player_event_subset.drop(["at_bat_count"], axis=1, inplace=True)
player_event_subset["stats_count"] = 1
stats_ts = (
    player_event_subset[["game_date", "stats_count"]]
    .groupby(["game_date"])
    .sum()
    .sort_index(ascending=True)
    .resample("1D")
    .mean()
    .reindex(t_index)
    .fillna(0)
)
display(stats_ts.tail())

combined_ts = stats_ts.merge(at_bat_ts, left_index=True, right_index=True)

def compute(window):
    num = combined_ts.loc[window.index, "stats_count"].sum()
    den = combined_ts.loc[window.index, "at_bat_count"].sum()
    # rookie debut safety consideration (in case of extremely low or high stat)
    if den < 10:
        return 0.1
    return num / (den + 1)

windows = combined_ts.rolling(365)
stat_ma = windows.apply(compute, raw=False)["stats_count"].shift()

# output will always be say ts starting from 2023-04-07, because data starts at 2022-04-07 (see above)
# this will be set in stone
computed_ma_start_date = stat_ma.index[365]
assert(date(computed_ma_start_date.year, computed_ma_start_date.month, computed_ma_start_date.day) == date(2023, 4, 7))
stat_ma_lst = stat_ma.tolist()[365:]
stat_ma_lst#.tail(30)


,at_bat_count
2023-04-13,0.0
2023-04-14,5.0
2023-04-15,5.0
2023-04-16,4.0
2023-04-17,5.0


,stats_count
2023-04-13,0.0
2023-04-14,1.0
2023-04-15,2.0
2023-04-16,0.0
2023-04-17,2.0


[0.13884785819793205,
 0.14032496307237813,
 0.1390532544378698,
 0.140117994100295,
 0.140117994100295,
 0.140117994100295,
 0.1394658753709199,
 0.1394658753709199,
 0.14094955489614244,
 0.14391691394658754,
 0.1426448736998514]

In [385]:
date(2023, 4, 7) - timedelta(365)

datetime.date(2022, 4, 7)

In [329]:
stat_ma_ind = (datetime(2023, 4, 17) - datetime(2023, 4, 8)).days
print(stat_ma_ind)
stat_ma_lst[stat_ma_ind]

9

In [373]:
ex_df = pd.DataFrame({"A": [[1, 10], [2, 20], [3, 30]]})
ex_df.loc[ex_df.index == 2, "A"].values[0]#[1]

[3, 30]

### Preprocessing Pipeline Planning

In [ ]:
# preprocessing
data["game_date"] = pd.to_datetime(data["game_date"])
data

In [ ]:
# get features
data["net_score"] = data["bat_score"] - data["fld_score"]
data["on_1b"] = np.where(data["on_1b"].notnull(), 1, 0)
data["on_2b"] = np.where(data["on_2b"].notnull(), 1, 0)
data["on_3b"] = np.where(data["on_3b"].notnull(), 1, 0)
data["days_since_start"] = data["game_date"].apply(lambda x: (x - start_of_season).days)
data

In [84]:
(date.today() - date(2023, 4, 21)).days

1

In [89]:
arr = [1, 2, 3, 7, 9]
window_size = 3
numbers_series = pd.Series(arr)
windows = numbers_series.rolling(window_size)
windows.mean().tolist()[window_size-1:]

[2.0, 4.0, 6.333333333333333]

In [91]:
data.events.value_counts()

events
field_out                    225
strikeout                    140
single                        90
walk                          65
double                        20
home_run                      19
grounded_into_double_play     16
force_out                     16
hit_by_pitch                  10
field_error                    6
sac_fly                        5
caught_stealing_2b             2
double_play                    2
catcher_interf                 1
Name: count, dtype: int64

In [442]:
# Testing code for the prod pipeline

sys.path.append('/Users/allenchen/projects/baseball-analytics/src')
from modeling.pipeline import data_pipeline
cols = [col for col in data.columns if col != "events"]
X, y = data[cols], data[["events"]]

In [443]:
%%time

Xt = data_pipeline.fit_transform(X, y)
transformed_feature_col_names = data_pipeline["feature_transformers"].get_feature_names_out().tolist()
features = pd.DataFrame(Xt, columns=transformed_feature_col_names)

CPU times: user 7.07 s, sys: 109 ms, total: 7.18 s
Wall time: 7.26 s


In [444]:
features_and_events = pd.concat([features, y], axis=1)
features_and_events.head()

,identity__batter,identity__pitcher,identity__game_date,identity__outs_when_up,identity__inning,identity__inning_topbot,encode_on_base_occupancy__on_1b,encode_on_base_occupancy__on_2b,encode_on_base_occupancy__on_3b,compute_net_score__net_score,compute_days_since_start__days_since_start,batter_365_days_ma__1B_perc,batter_365_days_ma__2B_perc,batter_365_days_ma__3B_perc,batter_365_days_ma__HR_perc,batter_365_days_ma__BB_perc,batter_365_days_ma__SO_perc,batter_365_days_ma__DP_perc,batter_365_days_ma__FO_perc,batter_365_days_ma__SF_perc,batter_365_days_ma__SH_perc,events
0,621493,676710,2023-04-17,2,9,Top,0,0,0,2,18,0.164274,0.036683,0.00319,0.038278,0.103668,0.210526,0.012759,0.401914,0.007974,0.0,so
1,660271,676710,2023-04-17,1,9,Top,0,0,0,2,18,0.142645,0.044577,0.008915,0.05052,0.095097,0.243685,0.013373,0.381872,0.004458,0.0,fo
2,621433,678394,2023-04-17,1,3,Top,1,1,0,4,18,0.0,0.0,0.0,0.0,0.6,0.2,0.0,0.0,0.0,0.0,so
3,687263,676710,2023-04-17,0,9,Top,0,0,0,2,18,0.0,0.0,0.0,0.0,0.0,0.222222,0.222222,0.444444,0.0,0.0,fo
4,621433,676710,2023-04-17,2,8,Top,0,0,0,2,18,0.0,0.0,0.0,0.0,0.6,0.2,0.0,0.0,0.0,0.0,so


In [445]:
date(2023, 4, 17) - date(2023, 4, 7)

datetime.timedelta(days=10)

In [ ]:
from sklearn import set_config

set_config(display='diagram')
display(clf_pipeline)

In [386]:
data.sort_values(by="game_date", ascending=True)

,game_date,batter,pitcher,events,description,game_type,stand,p_throws,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,game_pk,bat_score,fld_score
6574,2022-04-07,660271,664285,fo,hit_into_play,R,L,L,NaN,NaN,NaN,0,1,Bot,661042,0,0
6544,2022-04-07,622110,664285,s,hit_into_play,R,R,L,NaN,NaN,NaN,2,7,Bot,661042,0,1
6545,2022-04-07,543685,664285,dp,hit_into_play,R,R,L,NaN,NaN,545361.0,0,7,Bot,661042,0,1
6546,2022-04-07,545361,664285,e,hit_into_play,R,R,L,NaN,NaN,NaN,0,7,Bot,661042,0,1
6547,2022-04-07,660271,664285,fo,hit_into_play,R,L,L,NaN,NaN,NaN,2,6,Bot,661042,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,2023-04-17,621433,676710,so,swinging_strike,R,L,R,NaN,NaN,NaN,0,6,Top,718540,5,1
22,2023-04-17,681351,678394,s,hit_into_play,R,R,R,NaN,NaN,571875.0,2,1,Top,718540,4,0
21,2023-04-17,687263,676710,e,hit_into_play,R,R,R,NaN,NaN,NaN,1,6,Top,718540,5,1
19,2023-04-17,621433,678394,so,swinging_strike,R,L,R,NaN,571875.0,681351.0,2,1,Top,718540,4,0


### After generating features, remember to remove rows that are before the mv_start_date aka null values for mv

In [ ]:
"SO": "so",
    "DP": "dp",
    "FO": "fo",
    "SF": "sf",
    "SH": "sh",